https://github.com/c7blackjack/ADS509-TwitterScraper

In [1]:
#!pip install tweepy

In [2]:
!pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /Users/travis/opt/anaconda3/lib/python3.8/site-packages
Requires: oauthlib, requests, requests-oauthlib
Required-by: 


# Twitter API Pull

In [3]:
# These imports are for the Twitter section of the assignment
import tweepy
import os
import datetime
import re
from pprint import pprint

# These imports are for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

In [31]:
# Add any packages used here
import numpy as np
import math
import random

import urllib
import urllib.request

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called api_keys.py. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are not functional, so you'll need to get Twitter credentials and replace the placeholder keys.

In [5]:
from api_keys import api_key, api_key_secret, bearer_token

In [6]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is @37chandler. Feel free to use a different handle if you would like to look at someone else's data.

We will write code to explore a few aspects of the API:

1. Pull some of the followers @37chandler.
2. Explore response data, which gives us information about Twitter users.
3. Pull the last few tweets by @37chandler.

In [7]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users.

In [8]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 221, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1592, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 424.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2795, Followers: 3563.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2802, Followers: 3337.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers.

In [9]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14179, 'following_count': 2222, 'tweet_count': 56103, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the user_fields argument.

In [10]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [11]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for name we have John Chandler
for verified we have False
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 997, 'listed_count': 3}
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for id we have 33029025
for created_at we have 2009-04-18 22:08:22+00:00
for username we have 37chandler
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for location we have MN


Now a few questions for you about the user object.

Q: How many fields are being returned in the response object?

A: There are 9 fields being returned
    
Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.)

A: Yes Public Metrics is a Dictionary
    
Q: How many friends, followers, and tweets does this user have?

A: {Friends: 589, Followers: 193, Tweets: 99}

In [12]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569760631548690437
RT @dtmooreeditor: So there's a particular quirk of English grammar that I've always found quite endearing: the exocentric verb-noun compou…

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent bo

# Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using client.get_users_followers. This function has a parameter, max_results, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the pagination of our results.

The return object has the .data field, where the results will be found. It also has .meta, which we use to select the next "page" in the results using the next_token result. I will illustrate the ideas using our user from above.

## Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in this guide. We can make 15 user requests per 15 minutes, meaning that we can pull 4x15x100 = 60000 users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of handle.

In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.)

In [13]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=1000, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break
        

## Pulling Twitter Data for Your Artists

Now let's take a look at your artists and see how long it is going to take to pull all their followers.

In [14]:
artists = dict()

for handle in ['nfrealmusic','atmosphere'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 11.43 hours to pull all 685515 followers for nfrealmusic. 
It would take 3.49 hours to pull all 209489 followers for atmosphere. 


In [15]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called [artist name]_followers.txt in the "twitter" folder. For instance, for Cher I would create a file named cher_followers.txt. As you pull the data, also store it in an object like a list or a data frame.

In addition to creating a file that only has follower IDs in it, you will create a file that includes user data. From the response object please extract and store the following fields:

1. screen_name
2. name
3. id
4. location
5. followers_count
6. friends_count
7. description

Store the fields with one user per row in a tab-delimited text file with the name [artist name]_follower_data.txt. For instance, for Cher I would create a file named cher_follower_data.txt.

One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub.

In [16]:
num_followers_to_pull = 100*1000 # feel free to use this to limit the number of followers you pull.

In [17]:
print(os.getcwd())
os.chdir(os.getcwd() + '/twitter')

/Users/travis


In [18]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['atmosphere']

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

def convert_row(user):
    user_id = str(user.id)
    username = str(user.username)
    name = str(user.name)
    location = str(user.location)
    followers_count = str(user.public_metrics['followers_count'])
    following_count = str(user.public_metrics['following_count'])
    description = user.description
    description = description.replace("\n","|")
    description = description.replace("\t","	")
    description = description.replace('"','``')
    
    return '\t'.join([user_id,username,name,location,followers_count,following_count,description]) + "\n"



for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    user_obj = client.get_user(username=handle)

    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    

    for response in tweepy.Paginator(client.get_users_followers,
                                     user_obj.data.id,
                                     max_results=1000,
                                     limit=100,
                                     user_fields=['location','public_metrics','description']):
        print(response.meta)            
        for user in response.data : 
            follower_row = convert_row(user)

            user_data[handle].append(follower_row)
            followers_data[handle].append(user.id)
            user_id = str(user.id)
            f = open(followers_output_file,'a')
            f.write(follower_row)
            f.close()
            f = open(user_data_output_file,'a')
            f.write(user_id + '\t')
            f.close()

    

    # For each response object, extract the needed fields and store them in a dictionary or
    # data frame. 

    # I recommend writing your results for every response. This isn't the most efficient option
    # (since you're opening and closing the file regularly), but it ensures that your 
    # work is saved in case there is an issue with the API connection. 
    
    # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response


        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)

{'result_count': 1000, 'next_token': '1U7T01MV6FPHEZZZ'}
{'result_count': 1000, 'next_token': 'HTJBT9J3FIQHEZZZ', 'previous_token': 'P4MQ92Q9QC6EGZZZ'}
{'result_count': 1000, 'next_token': '19P3CBRFCI0HEZZZ', 'previous_token': 'IGPF8RAKJT5EGZZZ'}
{'result_count': 1000, 'next_token': '5BJL367MLPLHEZZZ', 'previous_token': 'JNTTU6UNK5VEGZZZ'}
{'result_count': 1000, 'next_token': 'V64C7DSLDDAHEZZZ', 'previous_token': 'UBJ2I3GGA6AEGZZZ'}
{'result_count': 1000, 'next_token': 'I8F6FBCHL0V1EZZZ', 'previous_token': '2SR6JQ4CJ2LEGZZZ'}
{'result_count': 1000, 'next_token': 'V5JBLKD944O1EZZZ', 'previous_token': '25TO94I9BF0UGZZZ'}
{'result_count': 1000, 'next_token': 'PA4E71T5S0LHEZZZ', 'previous_token': 'QE8F7BDPRV7UGZZZ'}
{'result_count': 1000, 'next_token': 'ST3LMPSCB4JHEZZZ', 'previous_token': 'LJO081Q847AEGZZZ'}
{'result_count': 1000, 'next_token': '5SNHHFP44OI1EZZZ', 'previous_token': 'LCOQ88POLFCEGZZZ'}
{'result_count': 1000, 'next_token': 'M5U6LU55E8BHEZZZ', 'previous_token': 'RHNJEJEPRJDU

Rate limit exceeded. Sleeping for 888 seconds.


{'result_count': 1000, 'next_token': 'N2ARF2F1UNVHCZZZ', 'previous_token': 'HBUGJK3Q6NPUGZZZ'}
{'result_count': 1000, 'next_token': 'U465DTF46VOHCZZZ', 'previous_token': '7L1I900G1C0EIZZZ'}
{'result_count': 1000, 'next_token': 'RR4HBTRQQBIHCZZZ', 'previous_token': 'NL6IA6GOQ47EIZZZ'}
{'result_count': 1000, 'next_token': 'H7M9QR1897C1CZZZ', 'previous_token': 'IP0SJ85V5KDEIZZZ'}
{'result_count': 1000, 'next_token': 'MP2QHQ2UMJ51CZZZ', 'previous_token': 'ALRAKALGMSJUIZZZ'}
{'result_count': 1000, 'next_token': 'U58OG96P1UVHCZZZ', 'previous_token': 'CIDM5N769GQUIZZZ'}
{'result_count': 1000, 'next_token': '8U40GRTFM6P1CZZZ', 'previous_token': '94CU2A0UVD0EIZZZ'}
{'result_count': 1000, 'next_token': 'IQLL8D75BQJ1CZZZ', 'previous_token': '45GHEGUOA16UIZZZ'}
{'result_count': 1000, 'next_token': '6498D7655QDHCZZZ', 'previous_token': 'UJRKURGDKPCUIZZZ'}
{'result_count': 1000, 'next_token': 'DNM2H7FSEA81CZZZ', 'previous_token': '5MQL9FQ7Q9IEIZZZ'}
{'result_count': 1000, 'next_token': '9L8STQ2K0I1H

Rate limit exceeded. Sleeping for 884 seconds.


{'result_count': 1000, 'next_token': '1P90639T28QHCZZZ', 'previous_token': 'ACRNTC3VSR0EIZZZ'}
{'result_count': 998, 'next_token': '845UD4VLK4N1CZZZ', 'previous_token': '81EQ3FO4UB5EIZZZ'}
{'result_count': 1000, 'next_token': '6EAQ5G95CGLHCZZZ', 'previous_token': '8B0I630NBV8UIZZZ'}
{'result_count': 1000, 'next_token': 'K5NPQUUE4KK1CZZZ', 'previous_token': 'HAHO9R8UJJAEIZZZ'}
{'result_count': 1000, 'next_token': '8ENBD9RI8CI1CZZZ', 'previous_token': 'NF97VV4HRBBUIZZZ'}
{'result_count': 1000, 'next_token': 'OB8MS8306GF1CZZZ', 'previous_token': 'KQPA95O3NNDUIZZZ'}
{'result_count': 1000, 'next_token': 'J503A0JARCD1CZZZ', 'previous_token': 'GTQV7QD0PFGUIZZZ'}
{'result_count': 1000, 'next_token': 'M387D1GBM4B1CZZZ', 'previous_token': 'GPEBDPRD4NIUIZZZ'}
{'result_count': 1000, 'next_token': 'L6EOGI0TCO91CZZZ', 'previous_token': 'GQACKMLB9VKUIZZZ'}
{'result_count': 1000, 'next_token': 'JUBVRPV9MK6HCZZZ', 'previous_token': '50K9LRFFJBMUIZZZ'}
{'result_count': 1000, 'next_token': '079BTGCFIC4HC

...

Rate limit exceeded. Sleeping for 886 seconds.


{'result_count': 1000, 'next_token': 'LB8UEHKH5VA1AZZZ', 'previous_token': 'GQ87RN7PV0KUKZZZ'}
{'result_count': 1000, 'next_token': '30T1446KGJ91AZZZ', 'previous_token': 'D4UPK84MQ0LUKZZZ'}
{'result_count': 1000, 'next_token': 'GUJVF9E8EN81AZZZ', 'previous_token': '3CV9H7D7FCMUKZZZ'}
{'result_count': 1000, 'next_token': '73FOUG5OQB6HAZZZ', 'previous_token': 'D9RH16JOH8NUKZZZ'}
{'result_count': 1000, 'next_token': '1LERKRG2G75HAZZZ', 'previous_token': 'C9EQ3Q5A5KPEKZZZ'}
{'result_count': 1000, 'next_token': 'C12BS28TC301AZZZ', 'previous_token': '1L24PBOKFSQEKZZZ'}
{'result_count': 1000, 'next_token': 'E4RVG1N6FQOHAZZZ', 'previous_token': '9COBOHJDKGVUKZZZ'}
{'result_count': 1000, 'next_token': 'JSM483BASMJ1AZZZ', 'previous_token': 'B7O8OF5OG97EKZZZ'}
{'result_count': 1000, 'next_token': '5P7V6CIVMEHHAZZZ', 'previous_token': '7TKRP5VT3TCUKZZZ'}
{'result_count': 999, 'next_token': 'FGN4022PMQGHAZZZ', 'previous_token': 'SJ10S7C59LEEKZZZ'}
{'result_count': 1000, 'next_token': 'K0M7S6BR8UG1A

Rate limit exceeded. Sleeping for 890 seconds.


{'result_count': 1000, 'next_token': '2SVDM8HG4A11AZZZ', 'previous_token': 'OE3LGKNN6TPUKZZZ'}


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [37]:
artists = {'lukeccombs':"https://www.azlyrics.com/l/lukecombs.html",
           'atmosphere':"https://www.azlyrics.com/a/atmosphere.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

Q: Take a look at the robots.txt page on www.azlyrics.com. (You can read more about these pages here.) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: In reading the disallow and allow parts, It would seem that it is allowed to scrape pages but not the db.

In [49]:

# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    thepage = urllib.request.urlopen(artist_page)
    soup = BeautifulSoup(thepage,"html.parser")

    for i in soup.find_all('div', attrs={'class' : 'listalbum-item'}):
        lyrics_pages[artist].append(i.a['href'])

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 

In [50]:
print(lyrics_pages)

defaultdict(<class 'list'>, {'lukeccombs': ['/lyrics/lukecombs/thewaysherides.html', '/lyrics/lukecombs/iknowsheaintready.html', '/lyrics/lukecombs/letthemoonshine.html', '/lyrics/lukecombs/shegotthebestofme.html', '/lyrics/lukecombs/sheriffyouwantto.html', '/lyrics/lukecombs/canigetanoutlaw.html', '/lyrics/lukecombs/outthere.html', '/lyrics/lukecombs/memoriesaremadeof.html', '/lyrics/lukecombs/lonelyone.html', '/lyrics/lukecombs/beercan.html', '/lyrics/lukecombs/hurricane.html', '/lyrics/lukecombs/onenumberaway.html', '/lyrics/lukecombs/donttemptme.html', '/lyrics/lukecombs/whenitrainsitpours.html', '/lyrics/lukecombs/thisonesforyou.html', '/lyrics/lukecombs/becarefulwhatyouwishfor.html', '/lyrics/lukecombs/igotawaywithyou.html', '/lyrics/lukecombs/honkytonkhighway.html', '/lyrics/lukecombs/houstonwegotaproblem.html', '/lyrics/lukecombs/mustvenevermetyou.html', '/lyrics/lukecombs/beautifulcrazy.html', '/lyrics/lukecombs/alongway.html', '/lyrics/lukecombs/shegotthebestofme.html', '/lyr

In [51]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [52]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For lukeccombs we have 72.
The full pull will take for this artist will take 0.2 hours.
For atmosphere we have 310.
The full pull will take for this artist will take 0.86 hours.


In [53]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [54]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [56]:
print(os.getcwd())
os.chdir(os.getcwd() + "/lyrics")
print(os.getcwd())


/Users/travis/twitter
/Users/travis/twitter/lyrics


In [101]:
## Code to test finding first instance of a div with no class and no id

thepage = requests.get('https://www.azlyrics.com/lyrics/lukecombs/thewaysherides.html')
soup = BeautifulSoup(thepage.content,"html.parser")

tag1 = soup.findAll('b')
tag2 = soup.findAll('div', attrs={'class': None, 'id': None})

print(tag1[1].extract())
print(tag2[0].extract())

<b>"The Way She Rides"</b>
<div>
<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->
Screamed into the driveway, with my foot stuck on the gas<br/>
Had the Crown mixed up in a Dixie cup, when she hopped into the cab<br/>
She said "Boy what you got in store, what we getting into tonight?"<br/>
Hell, I don't care, I'll go anywhere, just as long as you're by my side<br/>
<br/>
<i>[Chorus:]</i><br/>
Whether she's sitting shotgun with her hair done<br/>
Or with the seat leaned back with the radio on<br/>
Or the backstreets, in the backseat<br/>
Those baby blue eyes staring back at me<br/>
Whether it's in park or it's in drive<br/>
I love the way she rides<br/>
<br/>
So we hit that ground a running throwing caution into the wind<br/>
Well it wasn't long before that drink was gone, and she was shaking that ice again<br/>
She said find somewhere dark where we can park, and put a hurting on this bottle<br/>
So we 

In [85]:
print(os.getcwd())

/Users/travis/twitter/lyrics


In [106]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    #if os.path.isdir(artist) : 
        #shutil.rmtree(artist + "/")
    #os.mkdir(artist)
    # 2. Iterate over the lyrics pages
    for artist, lp in lyrics_pages.items():
        for i in lp:
            #print(os.getcwd() + "/" + artist + "/" + )
            path = url_stub + i
            r = requests.get(path)
            time.sleep(5 + 10*random.random())
        # 3. Request the lyrics page. 
            # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
            # to sleep after making the request
        # 4. Extract the title and lyrics from the page.
            soup = BeautifulSoup(r.content,"html.parser")

            title = soup.findAll('b')
            title = str(title[1])
            lyrics = soup.findAll('div', attrs={'class': None, 'id': None})
            lyrics = str(lyrics[0])
            
            path = os.getcwd()
            filename = generate_filename_from_link(i)

            f = open(os.getcwd() + "/" + artist + "/" + filename,'w')
            f.write(title)
            f.write('\n')
            f.write('\n')
            f.write(lyrics)
            f.close()
        

        
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    


In [107]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")


Total run time was 2.74 hours.


In [108]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

In [112]:
os.chdir('/Users/travis')

In [122]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: lukecombs and atmosphere.


In [124]:
print(artist_handles)
artist_handles.pop()
print(artist_handles)


['lukecombs', 'atmosphere', '.ipynb']
['lukecombs', 'atmosphere']


In [128]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    time.sleep(5 + 10*random.random())

    
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
#         print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
#         print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    


We see 108009 in your follower file for lukecombs, assuming a header row.
We have 10 data rows for lukecombs in the follower data file.
For lukecombs we have 0 unique locations.
For lukecombs we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------

We see 85097 in your follower file for atmosphere, assuming a header row.
We have 10 data rows for atmosphere in the follower data file.
For atmosphere we have 0 unique locations.
For atmosphere we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------



In [126]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For atmosphere we have 310 files.
For atmosphere we have roughly 193092 words, 10846 are unique.
For lukeccombs we have 71 files.
For lukeccombs we have roughly 26369 words, 2052 are unique.
